# Init Code

In [1]:
import Waven.WaveletGenerator as wg
import Waven.Analysis_Utils as au
import Waven.LoadPinkNoise as lpn
import numpy as np
import gc
import os
import torch

results = None

# Create Zebra Noise Stimulus (only once)

In [2]:
create_stim = False

if create_stim:
    import zebranoise

    #create a zebra noise video
    zebranoise.zebra_noise("fullscreen_zebra.mp4", xsize=800, ysize=600, tdur=60*15, fps=60, seed=7)

# see options below for more examples
#zebranoise.zebra_noise(
# "output2.mp4", 
# xsize=640, 
# ysize=480, 
# tdur=60*2, 
# levels=10, 
# xyscale=.2, 
# tscale=50, 
# fps=30, 
# xscale=1.0, 
# yscale=1.0, 
# seed=0)


# Create Gabor Library (only once)

#### Gabor Library Parameters:
- **N_thetas** (int): Number of orientations equally spaced between 0 and 180 degrees.
- **Sigmas** (list): Standard deviation of the Gabor filters expressed in pixels (radius of the Gaussian half peak width).
- **Frequencies** (list): Spatial frequencies expressed in pixels per cycle.
- **Phases** (list): 0 and π/2.
- **NX** (int): Number of azimuth positions (pixels) (x shape of the downsampled stimuli).
- **NY** (int): Number of elevation positions (pixels) (y shape of the downsampled stimuli).
- **Save Path** (string): Where to save the Gabor library.

In [3]:
# List of default parameters for the Gabor Library
gabor_param={
    "N_thetas":"6",
    "Sigmas": "[2, 3, 4, 5, 6, 8]",
    "Frequencies": "[0.015, 0.04, 0.07, 0.1]",
    "Phases": "[0, 90]",
    "NX": "100",
    "NY": "75",
    "Save Path":"/datajoint-data/data/leonk/analysis/zebra/gabor_library.npy"
}

In [4]:
# Generate Gabor library if it doesn't exist
lib_path = gabor_param["Save Path"]

if not os.path.exists(lib_path):
    print(f"Generating Gabor library at {lib_path}...")
    
    # Extract parameters
    n_thetas = int(gabor_param["N_thetas"])
    sigmas = np.array(eval(gabor_param["Sigmas"]))
    frequencies = np.array(eval(gabor_param["Frequencies"]))
    phases = np.array(eval(gabor_param["Phases"]))
    nx = int(gabor_param["NX"])
    ny = int(gabor_param["NY"])
    
    # Create coordinate arrays
    xs = np.arange(nx)
    ys = np.arange(ny)
    thetas = np.linspace(0, np.pi, n_thetas, endpoint=False)
    offsets = np.deg2rad(phases)  # Convert phases to radians
    
    print(f"Parameters:")
    print(f"  Grid: {nx}x{ny}")
    print(f"  Orientations: {n_thetas}")
    print(f"  Sigmas: {sigmas}")
    print(f"  Frequencies: {frequencies}")
    print(f"  Phases: {phases}°")
    print(f"\nThis may take several minutes...")
    
    # Generate library - this will take time!
    # Using the first frequency for the library
    library = wg.makeFilterLibrary(xs, ys, thetas, sigmas, offsets, frequencies[0], freq=True)
    
    # Save the library
    np.save(lib_path, library)
    print(f"\nGabor library saved! Shape: {library.shape}")
    print(f"File size: {os.path.getsize(lib_path) / (1024**2):.1f} MB")
else:
    print(f"Gabor library already exists at {lib_path}")
    library = np.load(lib_path)
    print(f"Library shape: {library.shape}")
    print(f"File size: {os.path.getsize(lib_path) / (1024**2):.1f} MB")

Gabor library already exists at /datajoint-data/data/leonk/analysis/zebra/gabor_library.npy
Library shape: (100, 75, 6, 6, 2, 7500)
File size: 7724.8 MB


# Run Zebra Analysis

Define Scans

In [5]:
scans = [
        'LE_ROS-2210_2025-11-28_scan9FXEU7TJ_sess9FXEU7TJ',
        'LE_ROS-2210_2025-11-30_scan9FXG17XS_sess9FXG17XS',
        'LE_ROS-2210_2025-12-02_scan9FXH819L_sess9FXH819L'
        ]

### Parameters Documentation

#### Alignment Parameters:
- **Dirs** (string): Where the raw data are located.
- **Experiment Info**: (mouse name, date, experiment number)
- **Number of Planes** (int): Number of acquisition planes.
- **Block End** (int): Timeframe where the experiment starts.
- **Number of Frames** (int): Number of frames (stim 30 Hz → 1800 frames/min).
- **Number of Trials to Keep** (int): Number of trials to keep.

#### Analysis Parameters:
- **screen_x**: Stimulus screen x size in pixels.
- **screen_y**: Stimulus screen y size in pixels.
- **NX** (int): Number of azimuth positions (pixels) (x shape of the downsampled stimuli).
- **NY** (int): Number of elevation positions (pixels) (y shape of the downsampled stimuli).
- **Resolution** (float): Microscope resolution (µm per pixel).
- **Sigmas** (list): Standard deviation of the Gabor filters expressed in pixels (radius of the Gaussian half peak width).
- **Visual Coverage** (list): [azimuth left, azimuth right, elevation top, elevation bottom] in visual degrees.
- **Analysis Coverage** (list): [azimuth left, azimuth right, elevation top, elevation bottom] in visual degrees.
- **Movie Path**: Path to the stimulus (.mp4).
- **Library Path**: Path to Gabor library (same as save path if ran).
- **Spks Path** (optional): Path to the spks.npy file to skip the alignment procedure. If set, ignores alignment parameters.

In [6]:
# List of parameters

param_defaults = {
    "Dirs": "/datajoint-data/data/leonk/",
    "Number of Planes": "1",
    "Block End": "0",
    "screen_x":"800",
    "screen_y":"600",
    "NX0": "100", # downsampled x positions from Gabor library
    "NY0": "75", # downsampled y positions from Gabor library
    "NX": "100", # target downsampled x positions for wavelet computation
    "NY": "75", # target downsampled y positions for wavelet computation
    "Resolution":"1.2",
    "Sigmas": "[2, 3, 4, 5, 6, 8]",
    "Frequencies": "[0.015, 0.04, 0.07, 0.1]",
    "Visual Coverage":"[-42, 42, 35, -15]",
    "Analysis Coverage": "[-42, 42, 35, -15]",
    "Number of Frames": "54000",  # stimulus frames in each trial
    "Number of Trials to Keep": "1",
    "Movie Path": "/datajoint-data/data/leonk/analysis/zebra/fullscreen_zebra.mp4",
    "Library Path": "/datajoint-data/data/leonk/analysis/zebra/gabor_library.npy",
    "Spks Path": "None",
    "Device": "cuda:3"
}


In [7]:
# Non-GUI analysis workflow
import matplotlib.pyplot as plt
from pathlib import Path

def run_analysis(param_defaults, gabor_param):
    """
    Run analysis without GUI - suitable for Jupyter notebooks and headless environments
    """
    
    # Extract parameters
    dirs = [param_defaults["Dirs"]]
    sigmas = np.array(eval(param_defaults["Sigmas"]))
    visual_coverage = eval(param_defaults["Visual Coverage"])
    analysis_coverage = eval(param_defaults["Analysis Coverage"])
    n_planes = int(param_defaults["Number of Planes"])
    block_end = int(param_defaults["Block End"])
    screen_x = int(param_defaults["screen_x"])
    screen_y = int(param_defaults["screen_y"])
    nx0 = int(param_defaults["NX0"])
    ny0 = int(param_defaults["NY0"])
    nx = int(param_defaults["NX"])
    ny = int(param_defaults["NY"])
    ns = len(sigmas)
    resolution = float(param_defaults["Resolution"])
    spks_path = param_defaults["Spks Path"]
    nb_frames = int(param_defaults["Number of Frames"])
    n_trial2keep = int(param_defaults["Number of Trials to Keep"])
    movpath = param_defaults["Movie Path"]
    lib_path = param_defaults["Library Path"]
    n_theta = int(gabor_param["N_thetas"])
    device = param_defaults["Device"]
    
    screen_ratio = abs(visual_coverage[0] - visual_coverage[1]) / nx
    xM, xm, yM, ym = analysis_coverage
    
    print(f"Visual coverage: {visual_coverage}, Sigmas: {sigmas}, NS: {ns}")
    print(f"Directories: {dirs}")

    #set device for torch
    torch.cuda.set_device(device)
    
    # Build paths
    pathdata = dirs[0] 
    pathsuite2p = pathdata + '/suite2p'
    
    deg_per_pix = abs(xM - xm) / nx
    sigmas_deg = np.trunc(2 * deg_per_pix * sigmas * 100) / 100
    
    print(f"Data path: {pathdata}")
    print(f"Suite2p path: {pathsuite2p}")
    
    # Load spike data
    if spks_path == 'None':
        print('Aligning data...')
        spks, spks_n, neuron_pos = lpn.loadSPKMesoscope(dirs[0], pathsuite2p, block_end, n_planes, nb_frames,
                                                        threshold=1.25, last=True, method='frame2ttl')
        neuron_pos = lpn.correctNeuronPos(neuron_pos, resolution)
        neuron_pos[:, 1] = abs(neuron_pos[:, 1] - np.max(neuron_pos[:, 1]))
    else:
        print(f'Loading spks file from {spks_path}')
        try:
            spks = np.load(spks_path)
            parent_dir = os.path.dirname(spks_path)
            neuron_pos = np.load(os.path.join(parent_dir, 'pos.npy'))
        except Exception as e:
            print(f"Error loading file: {e}")
            return None
    
    print(f"Spike data shape: {spks.shape}")
    print(f"Neuron positions shape: {neuron_pos.shape}")
    
    # Compute response reliability and skewness
    print("Computing response reliability and skewness...")
    
    # Check if we have multiple trials for reliability calculation
    n_neurons = spks.shape[0]
    if n_trial2keep > 1:
        # Multiple trials - compute cross-trial reliability
        respcorr = au.repetability_trial3(spks, neuron_pos, plotting=False)
    else:
        # Single trial - skip reliability, set to default values
        print("Single trial detected - skipping cross-trial reliability calculation")
        respcorr = np.ones(n_neurons)  # No filtering based on reliability for single trial
    
    skewness = au.compute_skewness_neurons(spks, plotting=False)
    skewness = np.array(skewness)
    
    # Create filter mask
    if n_trial2keep > 1:
        filter_mask = np.logical_and(respcorr >= 0.2, skewness <= 20)
    else:
        # For single trial, only filter by skewness
        filter_mask = skewness <= 20
    
    print(f"Neurons passing filter: {np.sum(filter_mask)}/{n_neurons}")
    
    # Plot neuron positions
    fig1, ax1 = plt.subplots(figsize=(8, 8))
    ax1.scatter(neuron_pos[:, 0], neuron_pos[:, 1], c='k', alpha=0.5, s=30)
    ax1.set_xlabel('X position (um)')
    ax1.set_ylabel('Y position (um)')
    ax1.set_title("Neuron Positions")
    plt.tight_layout()
    plt.show()
    
    # Load wavelets
    parent_dir = os.path.dirname(movpath)
    print(f"Loading wavelets from {parent_dir}...")
    
    # First try: load pre-computed downsampled wavelets
    try:
        wavelets_downsampled = np.load(os.path.join(parent_dir, 'dwt_downsampled_videodata.npy'))
        w_r_downsampled = wavelets_downsampled[0]
        w_i_downsampled = wavelets_downsampled[1]
        w_c_downsampled = wavelets_downsampled[2]
        del wavelets_downsampled
        gc.collect()
        print("Loaded downsampled wavelets")
    except Exception as e:
        print(f"Downsampled wavelets not found: {e}")
        
        # Second try: load coarse wavelets
        try:
            print("Attempting to load coarse wavelets...")
            w_r_downsampled, w_i_downsampled, w_c_downsampled = lpn.coarseWavelet(parent_dir, False, nx0, ny0, nx, ny,
                                                                                  n_theta, ns)
            print("Loaded coarse wavelets")
        except Exception as e:
            print(f"Error loading wavelets: {e}")
            
            # Third try: Check if downsampled video exists and generate wavelets
            downsampled_video_path = movpath[:-4] + '_downsampled.npy'
            if os.path.exists(downsampled_video_path):
                print(f"Found downsampled video at {downsampled_video_path}")
                print("Generating wavelet decomposition from downsampled video...")
                try:
                    videodata = np.load(downsampled_video_path)
                    print(f"Video data shape: {videodata.shape}")
                    
                    wg.waveletDecomposition(videodata, 0, sigmas, parent_dir, library_path=lib_path)
                    wg.waveletDecomposition(videodata, 1, sigmas, parent_dir, library_path=lib_path)
                    
                    w_r_downsampled, w_i_downsampled, w_c_downsampled = lpn.coarseWavelet(parent_dir, False, nx0, ny0, nx, ny,
                                                                                          n_theta, ns)
                    print("Completed wavelet decomposition from existing downsampled video")
                except Exception as e2:
                    raise RuntimeError(f"Error in wavelet decomposition from downsampled video: {e2}")
            else:
                # Fourth try: Full pipeline - downsample video then decompose
                print("Attempting full video processing pipeline...")
                try:
                    if (visual_coverage != analysis_coverage):
                        visual_coverage_arr = np.array(visual_coverage)
                        analysis_coverage_arr = np.array(analysis_coverage)
                        ratio_x = 1 - ((visual_coverage_arr[0] - visual_coverage_arr[1]) - (
                                analysis_coverage_arr[0] - analysis_coverage_arr[1])) / (
                                          visual_coverage_arr[0] - visual_coverage_arr[1])
                        ratio_y = 1 - ((visual_coverage_arr[2] - visual_coverage_arr[3]) - (
                                analysis_coverage_arr[2] - analysis_coverage_arr[3])) / (
                                          visual_coverage_arr[2] - visual_coverage_arr[3])
                    else:
                        ratio_x = 1
                        ratio_y = 1
                    
                    print(f"Downsampling video: {movpath}")
                    wg.downsample_video_binary(movpath, visual_coverage, analysis_coverage, shape=(ny, nx), chunk_size=1000,
                                            ratios=(ratio_x, ratio_y))
                    videodata = np.load(movpath[:-4] + '_downsampled.npy')
                    print(f"Downsampled video shape: {videodata.shape}")
                    
                    wg.waveletDecomposition(videodata, 0, sigmas, parent_dir, library_path=lib_path)
                    wg.waveletDecomposition(videodata, 1, sigmas, parent_dir, library_path=lib_path)
                    
                    w_r_downsampled, w_i_downsampled, w_c_downsampled = lpn.coarseWavelet(parent_dir, False, nx0, ny0, nx, ny,
                                                                                          n_theta, ns)
                    print("Completed full wavelet decomposition")
                except Exception as e3:
                    raise RuntimeError(f"Error in full pipeline: {e3}")
    
    # Compute receptive fields using Pearson correlation
    print("Computing receptive fields...")
    print(f"w_c_downsampled shape: {w_c_downsampled.shape}")
    print(f"Expected: (27300, 27, 11, {n_theta}, {ns})")
    
    # Use the actual number of frames we have
    n_frames_to_use = min(w_c_downsampled.shape[0], spks.shape[1])
    print(f"Using {n_frames_to_use} frames for RF calculation")
    
    rfs_gabor = au.PearsonCorrelationPinkNoise(w_c_downsampled[:n_frames_to_use].reshape(n_frames_to_use, -1), 
                                                np.mean(spks[:, :n_frames_to_use], axis=0),
                                                neuron_pos, nx, ny, ns, analysis_coverage, 
                                                screen_ratio, sigmas_deg, plotting=True)
    
    # Plot retinotopy maps
    fig2, ax2 = plt.subplots(2, 2, figsize=(14, 12))
    maxes1 = rfs_gabor[2]
    plt.rcParams['axes.facecolor'] = 'none'
    
    m = ax2[0, 0].scatter(neuron_pos[:, 0], neuron_pos[:, 1], s=10, c=maxes1[0], cmap='jet', alpha=filter_mask)
    fig2.colorbar(m, ax=ax2[0, 0])
    ax2[0, 0].set_title('Azimuth Preference (deg)')
    ax2[0, 0].set_xlabel('X (um)')
    ax2[0, 0].set_ylabel('Y (um)')
    
    m = ax2[0, 1].scatter(neuron_pos[:, 0], neuron_pos[:, 1], s=10, c=maxes1[1], cmap='jet_r', alpha=filter_mask)
    fig2.colorbar(m, ax=ax2[0, 1])
    ax2[0, 1].set_title('Elevation Preference (deg)')
    ax2[0, 1].set_xlabel('X (um)')
    ax2[0, 1].set_ylabel('Y (um)')
    
    m = ax2[1, 0].scatter(neuron_pos[:, 0], neuron_pos[:, 1], s=10, c=maxes1[2], cmap='hsv', alpha=filter_mask)
    fig2.colorbar(m, ax=ax2[1, 0])
    ax2[1, 0].set_title('Orientation Preference (deg)')
    ax2[1, 0].set_xlabel('X (um)')
    ax2[1, 0].set_ylabel('Y (um)')
    
    m = ax2[1, 1].scatter(neuron_pos[:, 0], neuron_pos[:, 1], s=10, c=maxes1[3], cmap='coolwarm', alpha=filter_mask)
    fig2.colorbar(m, ax=ax2[1, 1])
    ax2[1, 1].set_title('Preferred Size (deg)')
    ax2[1, 1].set_xlabel('X (um)')
    ax2[1, 1].set_ylabel('Y (um)')
    
    plt.tight_layout()
    plt.show()
    
    # Save results
    print("Saving results...")
    save_path = dirs + "/zebra/"
    np.save(os.path.join(save_path, 'correlation_matrix.npy'), rfs_gabor[0])
    np.save(os.path.join(save_path, 'maxes_indices.npy'), rfs_gabor[1])
    np.save(os.path.join(save_path, 'maxes_corrected.npy'), rfs_gabor[2])
    
    print("Analysis complete!")
    return {
        'spks': spks,
        'neuron_pos': neuron_pos,
        'rfs_gabor': rfs_gabor,
        'filter_mask': filter_mask,
        'respcorr': respcorr,
        'skewness': skewness
    }

In [ ]:
# Run the analysis for each scan
for scan in scans:
    scan_dir = "/datajoint-data/data/leonk/" + scan 
    param_defaults["Dirs"] = scan_dir
    print("Starting analysis pipeline for scan:", scan)
    results = run_analysis(param_defaults, gabor_param)

    if results is not None:
        print("Results keys:", results.keys())
        save_path = scan_dir + "/zebra/analysis_results.npy"
        np.save(save_path, results)

Starting analysis pipeline for scan: LE_ROS-2210_2025-11-28_scan9FXEU7TJ_sess9FXEU7TJ
Visual coverage: [-42, 42, 35, -15], Sigmas: [2 3 4 5 6 8], NS: 6
Directories: ['/datajoint-data/data/leonk/LE_ROS-2210_2025-11-28_scan9FXEU7TJ_sess9FXEU7TJ']
Data path: /datajoint-data/data/leonk/LE_ROS-2210_2025-11-28_scan9FXEU7TJ_sess9FXEU7TJ
Suite2p path: /datajoint-data/data/leonk/LE_ROS-2210_2025-11-28_scan9FXEU7TJ_sess9FXEU7TJ/suite2p
Aligning data...
last session
single plane
single plane
shape spks :  (564, 27300)
neuron_pos spks :  (564, 2)
Found 1 complete trials (54001 stimulus frames, 27300 imaging frames)
Processing trial 1
  Trial 1: 26963 imaging frames, 26963 timepoints
Detected incomplete final trial
Aligning trial 1: (564, 27300), max index: 27008
  26963 frames, 26963 timepoints, spks shape: (564, 26963)
Alignment complete: 1 trials processed
data aligned
Spike data shape: (1, 54000, 564)
Neuron positions shape: (564, 2)
Computing response reliability and skewness...
Single trial d

Wavelet transform (phase=1): 100%|██████████| 54000/54000 [00:51<00:00, 1056.57it/s]


loading wavelets...
downsampling


In [ ]:
# show the last results
plt.show()

# Check and visualize results

In [ ]:
#load results
if results is None:
    results = np.load('/home/leonk/waven-working-/temp/analysis_results.npy', allow_pickle=True).item()
#show result keys
print("Results keys:", results.keys())

Plot neuron positions

In [ ]:
%matplotlib inline
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt

xos = results["neuron_pos"][:,0]  # x position
yos = results["neuron_pos"][:,1]  # y position

print(f"x position range: {xos.min():.1f} to {xos.max():.1f}")
print(f"y position range: {yos.min():.1f} to {yos.max():.1f}")
print(f"Number of neurons: {len(xos)}")

# Create figure
fig, ax = plt.subplots(figsize=(8, 6))

# Calculate density for contours
xy = np.vstack([xos, yos])
kde = gaussian_kde(xy)

# Create grid for contour plot
x_grid = np.linspace(min(xos) - 2, max(xos) + 2, 100)
y_grid = np.linspace(min(yos) - 2, max(yos) + 2, 100)
X, Y = np.meshgrid(x_grid, y_grid)
positions = np.vstack([X.ravel(), Y.ravel()])
Z = kde(positions).reshape(X.shape)

# Plot density contours
contours = ax.contour(X, Y, Z, levels=5, colors='gray', alpha=0.5, linewidths=0.8)
ax.clabel(contours, inline=True, fontsize=8)

# Overlay scatter plot
ax.scatter(xos, yos, color='steelblue', s=50, alpha=0.7, edgecolor='black', linewidth=0.5)

ax.set_xlabel('x position', fontsize=11)
ax.set_ylabel('y position', fontsize=11)
ax.set_title('Neuron positions', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Plot RF positions

Looking at the code, results["rfs_gabor"] contains the output from PearsonCorrelationPinkNoise, which computes receptive fields. Based on how it's used in the plotting section, rfs_gabor is a tuple/list with 3 elements:

rfs_gabor[0] - Correlation matrix

Full correlation values between each neuron and each Gabor filter feature
Shape: (n_neurons, n_gabor_features) where n_gabor_features = 27 × 11 × n_theta × ns
These are the raw Pearson correlation coefficients
rfs_gabor[1] - Maximum indices

For each neuron, the indices of the Gabor features that gave the maximum correlation
Shape: (n_neurons, 4) where the 4 dimensions correspond to:
Azimuth position index (0-26)
Elevation position index (0-10)
Orientation index (0-7)
Size/scale index (0-5)
rfs_gabor[2] - Maxes corrected

The actual preference values converted to meaningful units:
maxes1[0]: Azimuth preference in degrees
maxes1[1]: Elevation preference in degrees
maxes1[2]: Orientation preference in degrees (0-180°)
maxes1[3]: Preferred size in degrees (sigma of Gabor)
Shape: (4, n_neurons)
These are the values displayed in the retinotopy maps
So in summary:

[0] = all correlations (for detailed analysis)
[1] = best-matching feature indices (for reconstruction)
[2] = preferred stimulus properties in degrees (for visualization)

In [ ]:
%matplotlib inline
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt

xos = results["rfs_gabor"][2][0]  # Azimuth preference in degrees
yos = results["rfs_gabor"][2][1]  # Elevation preference in degrees

print(f"Azimuth range: {xos.min():.1f} to {xos.max():.1f} degrees")
print(f"Elevation range: {yos.min():.1f} to {yos.max():.1f} degrees")
print(f"Number of neurons: {len(xos)}")

# Create figure
fig, ax = plt.subplots(figsize=(8, 6))

# Calculate density for contours
xy = np.vstack([xos, yos])
kde = gaussian_kde(xy)

# Create grid for contour plot
x_grid = np.linspace(min(xos) - 2, max(xos) + 2, 100)
y_grid = np.linspace(min(yos) - 2, max(yos) + 2, 100)
X, Y = np.meshgrid(x_grid, y_grid)
positions = np.vstack([X.ravel(), Y.ravel()])
Z = kde(positions).reshape(X.shape)

# Plot density contours
contours = ax.contour(X, Y, Z, levels=5, colors='gray', alpha=0.5, linewidths=0.8)
ax.clabel(contours, inline=True, fontsize=8)

# Overlay scatter plot
ax.scatter(xos, yos, color='steelblue', s=50, alpha=0.7, edgecolor='black', linewidth=0.5)

ax.set_xlabel('Azimuth (°)', fontsize=11)
ax.set_ylabel('Elevation (°)', fontsize=11)
ax.set_title('RF centers scatter', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
